In [1]:
import pandas as pd
import joblib
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from model.logistic_regression import logistic_regression
from model.decision_tree import decision_tree
from model.knn import k_nearest
from model.naive_bayes import naive_bayes
from model.random_forest_ensemble import random_forest
from model.xgboost_ensemble import xg_boost
from sklearn.metrics import (
    accuracy_score,
    roc_auc_score,
    precision_score,
    recall_score,
    f1_score,
    matthews_corrcoef,
    confusion_matrix
)

In [2]:
df = pd.read_csv("./data/HeartDiseaseTrain-Test.csv")

df = df.dropna() 

X = df.drop("target", axis=1)
y = df["target"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

In [3]:
num_features = [
    "age", "resting_blood_pressure", "cholestoral",
    "Max_heart_rate", "oldpeak"
]

cat_features = [
    "sex", "chest_pain_type", "fasting_blood_sugar",
    "rest_ecg", "exercise_induced_angina",
    "slope", "vessels_colored_by_flourosopy", "thalassemia"
]

preprocessor = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), num_features),
        ("cat", OneHotEncoder(handle_unknown="ignore"), cat_features)
    ]
)

In [4]:
def evaluate_model(model, X_test, y_test):
    """
    Evaluates a trained classification model on test data.

    Parameters:
    - model : trained sklearn Pipeline
    - X_test : pandas DataFrame (features)
    - y_test : pandas Series (true labels)

    Returns:
    - metrics : dict containing all evaluation metrics
    - cm : confusion matrix
    """

    # Always compute predictions
    y_pred = model.predict(X_test)

    # Compute AUC safely
    if hasattr(model, "predict_proba"):
        y_prob = model.predict_proba(X_test)[:, 1]
        auc = roc_auc_score(y_test, y_prob)

    elif hasattr(model, "decision_function"):
        scores = model.decision_function(X_test)
        auc = roc_auc_score(y_test, scores)

    else:
        auc = None  # very rare for classifiers

    # Metrics
    metrics = {
        "Accuracy": accuracy_score(y_test, y_pred),
        "AUC": auc,
        "Precision": precision_score(y_test, y_pred),
        "Recall": recall_score(y_test, y_pred),
        "F1": f1_score(y_test, y_pred),
        "MCC": matthews_corrcoef(y_test, y_pred)
    }

    # Confusion Matrix
    cm = confusion_matrix(y_test, y_pred)

    return metrics, cm

In [5]:
# Train model
logistic_regression = logistic_regression(preprocessor, X_train, y_train)

metrics, cm = evaluate_model(logistic_regression, X_test, y_test)

print(metrics)
print(cm)

# SAVE model (THIS LINE CREATES .pkl)
joblib.dump(logistic_regression, "./model/saved_models/logistic_regression.pkl")

{'Accuracy': 0.8731707317073171, 'AUC': 0.9444761904761905, 'Precision': 0.8558558558558559, 'Recall': 0.9047619047619048, 'F1': 0.8796296296296297, 'MCC': 0.7471136777897657}
[[84 16]
 [10 95]]


['./model/saved_models/logistic_regression.pkl']

In [6]:
# Train model
decision_tree = decision_tree(preprocessor, X_train, y_train)

metrics, cm = evaluate_model(decision_tree, X_test, y_test)

print(metrics)
print(cm)

# SAVE model (THIS LINE CREATES .pkl)
joblib.dump(decision_tree, "./model/saved_models/decision_tree.pkl")

{'Accuracy': 0.9853658536585366, 'AUC': 0.9857142857142858, 'Precision': 1.0, 'Recall': 0.9714285714285714, 'F1': 0.9855072463768116, 'MCC': 0.9711511393019859}
[[100   0]
 [  3 102]]


['./model/saved_models/decision_tree.pkl']

In [7]:
# Train model
k_nearest = k_nearest(preprocessor, X_train, y_train)

metrics, cm = evaluate_model(k_nearest, X_test, y_test)

print(metrics)
print(cm)

# SAVE model (THIS LINE CREATES .pkl)
joblib.dump(k_nearest, "./model/saved_models/knn.pkl")

{'Accuracy': 0.8634146341463415, 'AUC': 0.9698095238095238, 'Precision': 0.8468468468468469, 'Recall': 0.8952380952380953, 'F1': 0.8703703703703703, 'MCC': 0.7275282169269904}
[[83 17]
 [11 94]]


['./model/saved_models/knn.pkl']

In [8]:
# Train model
naive_bayes = naive_bayes(preprocessor, X_train, y_train)

metrics, cm = evaluate_model(naive_bayes, X_test, y_test)

print(metrics)
print(cm)

# SAVE model (THIS LINE CREATES .pkl)
joblib.dump(naive_bayes, "./model/saved_models/naive_bayes.pkl")

{'Accuracy': 0.824390243902439, 'AUC': 0.8707619047619047, 'Precision': 0.8108108108108109, 'Recall': 0.8571428571428571, 'F1': 0.8333333333333334, 'MCC': 0.6491863734758897}
[[79 21]
 [15 90]]


['./model/saved_models/naive_bayes.pkl']

In [9]:
# Train model
random_forest = random_forest(preprocessor, X_train, y_train)

metrics, cm = evaluate_model(random_forest, X_test, y_test)

print(metrics)
print(cm)

# SAVE model (THIS LINE CREATES .pkl)
joblib.dump(random_forest, "./model/saved_models/random_forest.pkl")

{'Accuracy': 1.0, 'AUC': 1.0, 'Precision': 1.0, 'Recall': 1.0, 'F1': 1.0, 'MCC': 1.0}
[[100   0]
 [  0 105]]


['./model/saved_models/random_forest.pkl']

In [10]:
# Train model
xg_boost = xg_boost(preprocessor, X_train, y_train)

metrics, cm = evaluate_model(xg_boost, X_test, y_test)

print(metrics)
print(cm)

# SAVE model (THIS LINE CREATES .pkl)
joblib.dump(random_forest, "./model/saved_models/xg_boost.pkl")

{'Accuracy': 1.0, 'AUC': 1.0, 'Precision': 1.0, 'Recall': 1.0, 'F1': 1.0, 'MCC': 1.0}
[[100   0]
 [  0 105]]


/Library/Frameworks/Python.framework/Versions/3.14/lib/python3.14/site-packages/xgboost/training.py:199: UserWarning: [08:25:58] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


['./model/saved_models/xg_boost.pkl']